In [ ]:
LAB_USER_NAME = 'DEMO'

**Important**: To initialise this notebook, edit the cell above to set `LAB_USER_NAME` to your name, then click **Run->Run All Cells** in the top menu bar. This needs to be the same as the name used in **Part 1** for the files from that part to be loaded.

In [ ]:
import panel as pn
pn.extension()
import sys
import os

# add inline dashboard libraries to path so they can be imported later
sys.path.append('../../../dashboards-inline')

LAB_DIR = os.path.join('/home/data/', LAB_USER_NAME)
os.makedirs(LAB_DIR, exist_ok=True)
print('User data directory:', LAB_DIR)

## Part Three: The Spin Echo Sequence

### 1. Overview

In this part we will explore the spin echo effect using two sequential RF pulses, and seeing how the echo changes when changing the spacing between the RF pulses and spoiling the homogeneity of the magnetic field.

The initial shims will be loaded from your user data directory (displayed above), which should contain a `shims.yaml` file generated by running the autoshim tool in **Part 1**. 

### 2. Theory

Spin-spin relaxation causes the NMR signal to decay exponentially via dephasing with a time constant called $T_2$, described by the following equation:

$$\large M(t) = M(0) \cdot \mathrm{e}^{-t/T_2}$$

### 3. Echo Time and Spin-Spin ($T_2$) Relaxation
The dashboard below allows you to modify the echo time and see the effect on the Spin Echo signal and spectrum. For this pulse sequence the signal is acquired the entire time from the first RF pulse, so you will be able to see the FID signal and glitches caused by the RF  pulses, as well as the echo signal. Usually only the echo signal would be acquired with an acquisition window centred on the echo time.

#### **Plot Tool Notes**
The $y$ value can be easily read off a plot line by hovering over it with the mouse, as long as the hover tool is enabled.

#### **Step One**
Try varying the echo time and running the Spin Echo experiment to see how the echo moves. The pulse sequence is automatically positioning the second RF pulse at half the echo time after the first in order to have the echo appear at the desired point in time. With this experiment you can see the position of the second pulse as it causes a large, but very short, glitch in the signal.

#### **Step Two**
Run the Spin Echo experiment at a short and long echo time and measure the maximum amplitude magnitude at each. Use the equation given in section 2 to estimate $T_2$

Hint: Write the formula for $\large \frac{M(t_1)}{M(t_2)}$, where $t_1$ and $t_2$ are your chosen echo times.

In [ ]:
# load user shims file
import yaml
SHIM_ORDER = ['shim_x', 'shim_y', 'shim_z', 'shim_z2', 'shim_zx', 'shim_zy', 'shim_xy', 'shim_x2y2']
SHIM_FILE = os.path.join(LAB_DIR, 'shims.yaml')
if not os.path.isfile(SHIM_FILE):
    raise Exception(f'Shim file not found at {SHIM_FILE}! Ensure LAB_USER_NAME is set correctly and Part 1 has been completed.')

with open(SHIM_FILE, 'r') as f:
    SHIM_INIT = yaml.safe_load(f)

# create shim inputs, using saved user shims as initial values
shim_inputs = {}
for shim_key in SHIM_ORDER:
    shim_name = shim_key.split('_')[-1].upper()
    shim_inputs[shim_key] = pn.widgets.FloatInput(name=shim_name, start=-1, end=1, step=0.01, value=round(SHIM_INIT[shim_key], 2), width=80)

# button to reset shim inputs to the saved values
reset_btn = pn.widgets.Button(name='Reset', button_type='primary', align='end', width=100)

def reset_shim_inputs(e):
    for shim_key in SHIM_ORDER:
        shim_inputs[shim_key].value = round(SHIM_INIT[shim_key], 2)

reset_btn.on_click(reset_shim_inputs)


# input and buttons to save and load shim inputs
shim_save_name = pn.widgets.TextInput(name='', placeholder='Enter a filename, e.g. bad_x_shim')
shim_save_btn = pn.widgets.Button(name='Save', button_type='primary', width=100)
shim_load_btn = pn.widgets.Button(name='Load', button_type='primary', width=100)

def save_shims(e):
    if shim_save_name.value == '':
        return
    with open(os.path.join(LAB_DIR, f'{shim_save_name.value}.yaml'), 'w') as f:
        yaml.dump({k: v.value for k, v in shim_inputs.items()}, f) # extract values of inputs and save

def load_shims(e):
    if shim_save_name.value == '':
        return
    with open(os.path.join(LAB_DIR, f'{shim_save_name.value}.yaml'), 'r') as f:
        loaded_shims = yaml.safe_load(f)
        for shim_key in SHIM_ORDER:
            shim_inputs[shim_key].value = loaded_shims[shim_key]

shim_save_btn.on_click(save_shims)
shim_load_btn.on_click(load_shims)

# echo time input
echo_time = pn.widgets.FloatInput(name='Echo Time (seconds)', start=0.02, end=0.2, step=0.01, value=0.1, width=200)
    
from full_acq_SE import FullAcqSEApp # from dashboards-inline directory that was added to sys.path
# set some parameters directly
override_pars = dict(
    t_echo=echo_time,
    n_scans=1,
    n_samples=20000,
    t_dw=10e-6,
    t_end=1.0 # relatively long end time to avoid T1 effects when trying to calculate T2
)

# add shim inputs
override_pars.update(shim_inputs)

# create dashboard app
section3_app = FullAcqSEApp(
    override_pars=override_pars,
    show_magnitude=True,
    show_complex=True
)

# display layout
pn.Column(
    echo_time,
    pn.Row(*([shim_inputs[shim_key] for shim_key in SHIM_ORDER]+[reset_btn])), # take shim inputs in order and concatenate reset button
    pn.Row(shim_save_name, shim_save_btn, shim_load_btn),
    section3_app.main(),
    sizing_mode='stretch_width'
)

### 4. Spin Echo Signal with Spoiled Homogeneity

The dashboard below allows you to modify the shim values and see the effect on the Spin Echo signal and spectrum. Try adjusting the shim values in increments of 0.01, which can be done with the scroll wheel after clicking inside the input box, and running the experiment. *Run Loop* can also be used to automatically acquire the Spin Echo repeatedly.

#### **Step One**
Run the spin echo acquisition with the loaded shims from Part 1 (use the *Reset* button if necessary).

#### **Step Two**
Spoil the field by adjusting the shim values, or loading a bad shim file from Part 2. What effect does spoiling the field have on the maximum amplitude of the signal? And what effect on the height of the spectrum peak?

Note: With long echo times, diffusion may also cause a reduction in signal that is dependent on homogeneity.

In [ ]:
from SE import SEApp # from dashboards-inline directory that was added to sys.path
# create SE experiment using shim inputs
# set some parameters directly
override_pars = dict(
    t_echo=echo_time,
    n_scans=1,
    n_samples=1800,
    t_dw=10e-6, # using a long dwell time for narrow bandwith to more easily see the spectrum shape
    t_end=0.2
)

# add shim inputs
override_pars.update(shim_inputs)

# create dashboard app
section4_app = SEApp(
    override_pars=override_pars,
    enable_run_loop=True,
    show_magnitude=True
)

# display layout
pn.Column(
    echo_time,
    pn.Row(*([shim_inputs[shim_key] for shim_key in SHIM_ORDER]+[reset_btn])), # take shim inputs in order and concatenate reset button
    pn.Row(shim_save_name, shim_save_btn, shim_load_btn),
    section4_app.main(),
    sizing_mode='stretch_width'
)